In [9]:
import os 
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import google.generativeai as genai

In [25]:
load_dotenv()
api_key=os.getenv('GEMINI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [12]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [13]:
ed = Website("https://edwarddonner.com")
print(ed.title)
print("+++++++  TEXT +++++++",ed.text)

Home - Edward Donner
+++++++  TEXT +++++++ Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of 

In [16]:
system_prompt = """You are an assistant that analyzes the contents of a website
and provides a short summary, ignoring text that might be navigation related.
Respond in markdown."""

In [17]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += """\nThe contents of this website is as follows;
please provide a short summary of this website in markdown.
If it includes news or announcements, then summarize these too.\n\n"""
    user_prompt += website.text
    return user_prompt

In [18]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows;
please provide a short summary of this website in markdown.
If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

In [37]:
def messages_for(website):
    return f"{system_prompt} {user_prompt_for(website)}"
    

In [32]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [22]:
messages_for(ed)

'You are an assistant that analyzes the contents of a website\nand provides a short summary, ignoring text that might be navigation related.\nRespond in markdown. You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows;\nplease provide a short summary of this website in markdown.\nIf it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potenti

In [38]:
def summarize(url):
    website = Website(url)
    genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

    model = genai.GenerativeModel("gemini-2.5-flash")

    response = model.generate_content(messages_for(website))
    return response.text

In [39]:
summarize("https://edwarddonner.com")


E0000 00:00:1759935452.920961   20769 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


'This website is the personal homepage of Ed Donner, co-founder and CTO of Nebula.io, an AI company focused on talent discovery and recruitment. He previously founded untapt, acquired in 2021. Ed expresses his passion for writing code, experimenting with LLMs, DJing, and amateur electronic music production.\n\n**News/Announcements:**\n\n*   **September 15, 2025:** AI in Production: Gen AI and Agentic AI on AWS at scale\n*   **May 28, 2025:** Connecting my courses – become an LLM expert and leader\n*   **May 18, 2025:** 2025 AI Executive Briefing\n*   **April 21, 2025:** The Complete Agentic AI Engineering Course'